### Name :- Rajat Jain
### UTA ID :- 1001753645


In [1]:
# cleaning texts 
import pandas as pd 
import re 
import nltk 
from nltk.corpus import stopwords 
from nltk.stem.porter import PorterStemmer 
from sklearn.feature_extraction.text import CountVectorizer 

# dataset = [["I liked the movie", "positive"], 
# 		["It’s a good movie. Nice story", "positive"], 
# 		["Hero’s acting is bad but heroine looks good.\ 
# 			Overall nice movie", "positive"], 
# 			["Nice songs. But sadly boring ending.", "negative"], 
# 			["sad movie, boring movie", "negative"]] 
			
# dataset = pd.DataFrame(dataset) 
# dataset.columns = ["Text", "Reviews"] 


### Data cleaning and Splitting.
### I have adjusted the rating values by rounding up to the nearest integer

In [2]:
df = pd.read_csv('bgg-15m-reviews.csv')
df = df[df['comment'].notna()]
df['rating'] = df['rating'].round(0)
df['rating'] = df['rating'].astype(int)
df = df[['comment','rating']]

In [3]:
df

,comment,rating
1,Hands down my favorite new game of BGG CON 200...,10
2,I tend to either love or easily tire of co-op ...,10
4,This is an amazing co-op game. I play mostly ...,10
5,Hey! I can finally rate this game I've been pl...,10
8,Love it- great fun with my son. 2 plays so far...,10
...,...,...
15823242,KS Collector's Bundle with a friend of mine,10
15823247,Belekokio Gerumo...,10
15823253,Excelente!! lo mejor que probé.,10
15823264,Turn based preview looks very promising. The g...,8


In [4]:
from sklearn.model_selection import StratifiedShuffleSplit
sss = StratifiedShuffleSplit(n_splits=11, test_size=0.99, random_state=0)
sss.get_n_splits(df['comment'], df['rating'])
for train_index, test_index in sss.split(df['comment'], df['rating']):
    train_set = df.iloc[train_index]
    test_set = df.iloc[test_index]

In [5]:
train_set

,comment,rating
1740396,Archived,6
6636962,Un must have dans le genre.,9
1159113,"Excellent game, very thematic. Luck of the dr...",8
4726306,Great game for new gamers! Check out our doubl...,7
9668014,2007,8
...,...,...
4120186,1–4 Players 60–150 Min,9
10254983,I'm absolutely in love with this game. I've p...,9
4941591,Ben,7
7558943,"Un astratto molto divertente ed ""educativo"". C...",7


In [6]:
test_set

,comment,rating
2172215,Play with 2-4,7
14860516,Star Fleet Battles light.,7
3252395,Fun strategy game. Light enough to play with n...,8
14120044,OMFG. Most disappointing purchase ever. This ...,3
260031,Essentiel,9
...,...,...
13960695,Solid solitaire fun!,8
6792273,I always liked Sub Search as opposed to Battle...,5
12233492,The new characters are nice additions. Good ex...,9
10876117,P2,7


In [7]:
from sklearn.model_selection import StratifiedShuffleSplit
sss = StratifiedShuffleSplit(n_splits=11, test_size=0.99, random_state=0)
sss.get_n_splits(test_set['comment'], test_set['rating'])
for train_index, test_index in sss.split(df['comment'], df['rating']):
    test_set = df.iloc[train_index]

In [8]:
test_set

,comment,rating
1740396,Archived,6
6636962,Un must have dans le genre.,9
1159113,"Excellent game, very thematic. Luck of the dr...",8
4726306,Great game for new gamers! Check out our doubl...,7
9668014,2007,8
...,...,...
4120186,1–4 Players 60–150 Min,9
10254983,I'm absolutely in love with this game. I've p...,9
4941591,Ben,7
7558943,"Un astratto molto divertente ed ""educativo"". C...",7


### I have used Multinomial Naive Bayes and SVM
### The parameters of SVM kernels are changed. I had tested it for kenrel = 3 and kernel = 10.

# NOTE :- I cleared the outputs by mistake before submitting and the training process takes time so there is no output for SVM  but I am attaching the model files of both naive bayes and SVM as a proof.

In [9]:
nltk.download('stopwords') 

corpus = [] 

for i in range(0, len(train_set)): 
	text = re.sub('[^a-zA-Z]', '', train_set.iloc[i]['comment']) 
	text = text.lower() 
	text = text.split() 
	ps = PorterStemmer() 
	text = ''.join(text) 
	corpus.append(text) 

# creating bag of words model 
cv = CountVectorizer(max_features = 5000) 

X = cv.fit_transform(corpus).toarray() 
y = train_set.rating.values 


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Rajat\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [10]:
# splitting the data set into training set and test set 
from sklearn.model_selection import train_test_split 

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.20, random_state = 0) 


In [11]:
# fitting naive bayes to the training set 
from sklearn.metrics import accuracy_score
from sklearn.naive_bayes import MultinomialNB 
from sklearn.metrics import confusion_matrix 

classifier = MultinomialNB(); 
classifier.fit(X_train, y_train) 

# predicting test set results 
y_pred = classifier.predict(X_test) 

# making the confusion matrix 
cm = confusion_matrix(y_test, y_pred)
print("Multinomial Accuracy Score -> ",accuracy_score(y_pred, y_test)*100)


Multinomial Accuracy Score ->  24.724540901502504


In [ ]:
# from joblib import dump, load
# dump(classifier, 'naive_model.joblib') 


In [ ]:
# dump(cv,'naive_cv.joblib')

In [ ]:
# from sklearn.metrics import classification_report
# print(classification_report(y_pred, y_test))

In [ ]:
from sklearn import svm
from sklearn.metrics import accuracy_score
SVM = svm.SVC(C=1.0, kernel='poly', degree=3, gamma='auto')
SVM.fit(X_train,y_train)
# predict the labels on validation dataset
predictions_SVM = SVM.predict(X_test)
# Use accuracy_score function to get the accuracy

In [ ]:
from sklearn import svm
from sklearn.metrics import accuracy_score
SVM = svm.SVC(C=1.0, kernel='poly', degree=10, gamma='auto')
SVM.fit(X_train,y_train)
# predict the labels on validation dataset
predictions_SVM = SVM.predict(X_test)
# Use accuracy_score function to get the accuracy


In [ ]:
print("SVM Accuracy Score for degree 10 -> ",accuracy_score(predictions_SVM, y_test)*100)

In [ ]:
# import pickle
# s = pickle.dumps(classifier)

In [ ]:
# from joblib import dump, load
# dump(SVM, 'svm_model.joblib') 

In [ ]:
# from joblib import dump, load
# cl = load('naive_model.joblib') 
# cv = load('naive_cv.joblib')

### As a proof of SVM I am loading this model and making a prediction of the sample comment which is stored in the variable "ji"

In [17]:
from joblib import dump, load
clf = load('svm_model.joblib') 
l = load('naive_cv.joblib')

In [18]:
import re
from nltk.stem.porter import PorterStemmer
from sklearn.feature_extraction.text import CountVectorizer
ji = "great work. well played"
c = []
text = re.sub('[^a-zA-Z]', '', ji) 
text = text.lower() 
text = text.split() 
ps = PorterStemmer() 
text = ''.join(text) 
c.append(text) 
ji = l.transform(c).toarray()

In [19]:
p =clf.predict(ji)

In [20]:
p

array([8])

### The accuracy of Multinomial naive bayes is better than SVM hence I have selected the naive bayes model. The problems I faced are in data cleaning. In order to increase the accuracy I have used Stemming.
### I have even tested for 2 hyperparameters of SVM and kernel = 3 I get an accuracy of around 9% and for kernel = 10 I get an accuracy of around 24 %

### References :- 
### https://www.geeksforgeeks.org/applying-multinomial-naive-bayes-to-nlp-problems/
### https://medium.com/@bedigunjit/simple-guide-to-text-classification-nlp-using-svm-and-naive-bayes-with-python-421db3a72d34
### https://scikit-learn.org/stable/modules/model_persistence.html